GitHub Repository for this project can be found at https://github.com/seel6470/CSPB-3202-Final-Project

# Beating Super Mario Bros 1-1 With Reinforcement Learning
<ol type="I">
<li><a href='#short-overview'>Short Overview</a></li>
<li><a href='#approach'>Approach</a>
  <ol type="A">
    <li><a href='#random-agent'>Random Agent</a></li>
    <li><a href='#heuristic-agent'>Heuristic Agent</a></li>
    <li><a href='#ppo-agent'>PPO Agent</a></li>
    <li><a href='#q-learning-agent'>Q-Learning Agent</a></li>
  </ol>
</li>
<li><a href='#results'>Results</a></li>
<li><a href='#conclusion'>Conclusion</a></li>
<li><a href='#references'>References</a></li>
</ol>

<a id='short-overview'></a>
## Short Overview

*short overview of what your project is about (e.g. you're building /testing certain RL models in certain environments; yes you can test your algorithm in more than 1 environment if your goal is to test an algorithm(s) performances in different settings)*

1. Does it include the clear overview on what the project is about? (4)

2. Does it explain how the environment works and what the game rules are? (4)

For my project, I chose to teach a learning model to play the original Super Mario Bros. game for the NES. I utilized a library created by Christian Kauten called gym-super-mario-bros, which provides an OpenAI Gym environment using the nes-py emulator (Kauten, 2018). The challenge is to beat as many levels as possible in the original Mario game for NES with the following rules of the game.

The goal of the game is to avoid enemies and pits to reach the end of each level. One hit and Mario loses a life, starting over from the nearest checkpoint. Power-ups provide Mario an additional hit. The following page from the original game manual outlines the inputs Mario receives for the game:

![image](images/controls.jpg)

Nintendo. (1985). Super Mario Bros. Instruction Manual. Nintendo of America Inc. Retrieved from [https://www.nintendo.co.jp/clv/manuals/en/pdf/CLV-P-NAAAE.pdf]

The game environment takes these controls and creates the following action lists that can be used within the environment wrapper:

```python
# actions for the simple run right environment
RIGHT_ONLY = [
    ['NOOP'],
    ['right'],
    ['right', 'A'],
    ['right', 'B'],
    ['right', 'A', 'B'],
]


# actions for very simple movement
SIMPLE_MOVEMENT = [
    ['NOOP'],
    ['right'],
    ['right', 'A'],
    ['right', 'B'],
    ['right', 'A', 'B'],
    ['A'],
    ['left'],
]


# actions for more complex movement
COMPLEX_MOVEMENT = [
    ['NOOP'],
    ['right'],
    ['right', 'A'],
    ['right', 'B'],
    ['right', 'A', 'B'],
    ['A'],
    ['left'],
    ['left', 'A'],
    ['left', 'B'],
    ['left', 'A', 'B'],
    ['down'],
    ['up'],
]
```

The environment can also determine the following keys for the gamestate:

| Key       | Type | Description                                |
|-----------|------|--------------------------------------------|
| coins     | int  | The number of collected coins              |
| flag_get  | bool | True if Mario reached a flag or ax         |
| life      | int  | The number of lives left, i.e., {3, 2, 1}  |
| score     | int  | The cumulative in-game score               |
| stage     | int  | The current stage, i.e., {1, ..., 4}       |
| status    | str  | Mario's status, i.e., {'small', 'tall', 'fireball'} |
| time      | int  | The time left on the clock                 |
| world     | int  | The current world, i.e., {1, ..., 8}       |
| x_pos     | int  | Mario's x position in the stage (from the left) |
| y_pos     | int  | Mario's y position in the stage (from the bottom) |

Additionally, the environment utilizes the following parameters for the reward function:

1. **v**: the difference in agent x values between states
   - In this case, this is instantaneous velocity for the given step
   - \( v = x1 - x0 \)
     - \( x0 \) is the x position before the step
     - \( x1 \) is the x position after the step
   - Moving right \( \implies v > 0 \)
   - Moving left \( \implies v < 0 \)
   - Not moving \( \implies v = 0 \)

2. **c**: the difference in the game clock between frames
   - The penalty prevents the agent from standing still
   - \( c = c0 - c1 \)
     - \( c0 \) is the clock reading before the step
     - \( c1 \) is the clock reading after the step
   - No clock tick \( \implies c = 0 \)
   - Clock tick \( \implies c < 0 \)

3. **d**: a death penalty that penalizes the agent for dying in a state
   - This penalty encourages the agent to avoid death
   - Alive \( \implies d = 0 \)
   - Dead \( \implies d = -15 \)

\( r = v + c + d \)

The reward is clipped into the range \([-15, 15]\).

In addition to the gym-super-mario-bros library, nes-py is required in order to emulate the NES in Python. Huge thanks to Kautenja for creating the environment and making this project possible! (Kauten, 2018)

<a id='approach'></a>

## Approach

*explain your environment, your choice of model(s), the methods and purpose of testing and experiments, explain any trouble shooting required.*

3. Does it explain clearly the model(s) of choices, the methods and purpose of tests and experiments? (7)

4. Does it show problem solving procedure- e.g. how the author solved and improved when an algorithm doesn't work well. Note that it's not about debugging or programming/implementation, but about when a correctly implemented algorithm wasn't enough for the problem and the author had to modify/add some features or techniques, or compare with another model, etc. (7)

The initial setup for the environment was a bit tricky due to some incompatibilities between the chosen gym library gym-super-mario-bros JoypadSpace wrapper and the current version of OpenAi's gym framework, specifically with the `reset` method. Huge thanks to NathanGavinski who supplied [a workaround](https://github.com/Kautenja/gym-super-mario-bros/issues/128#issuecomment-1954019091) in the issues forum for gym-super-mario-bros Git. (NathanGavenski, 2023).

The following code utilizes this fix along with the suggested boilerplate setup from the gym-super-mario-bros documentation:

In [35]:
import gym
import gym_super_mario_bros
import time
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
from gymnasium.wrappers import StepAPICompatibility, TimeLimit

# Create the Super Mario Bros. environment
env = gym.make('SuperMarioBros-v0')
steps = env._max_episode_steps  # get the original max_episode_steps count

# Set the Joypad wrapper
env = JoypadSpace(env.env, SIMPLE_MOVEMENT)

# Define a new reset function to accept `seeds` and `options` args for compatibility
def gymnasium_reset(self, **kwargs):
    return self.env.reset(), {}

# Overwrite the old reset to accept `seeds` and `options` args
env.reset = gymnasium_reset.__get__(env, JoypadSpace)

# Set TimeLimit back
env = TimeLimit(StepAPICompatibility(env, output_truncation_bool=True), max_episode_steps=steps)

<a id='random-agent'></a>

## Random Agent

Let's create an agent that makes random movements just to make sure our environment is working:

In [36]:
done = True
max_x = 0
max_world = 0
max_stage = 0
# Run the environment for 5000 steps
for step in range(1000):
    if done:
        state = env.reset()
    state, reward, done, truncated, info = env.step(env.action_space.sample())
    if info['world'] > max_world:
        max_world = info['world']
        max_x = 0
        max_stage = 1
    elif info['stage'] > max_stage:
        max_stage = info['stage']
        max_x = 0
    elif info['world'] == max_world and info['stage'] == max_stage and info['x_pos'] > max_x:
        max_x = info['x_pos']
    done = done or truncated
    env.render()
    time.sleep(0.01)  # Add a delay of 0.05 seconds between frames

# Close the environment
env.close()
print(f"Max X: {max_x}")
print(f"Max World: {max_world}")
print(f"Max Stage: {max_stage}")

c:\Users\sethe\AppData\Local\Programs\Python\Python312\Lib\site-packages\gym\utils\passive_env_checker.py:272: UserWarning: WARN: No render modes was declared in the environment (env.metadata['render_modes'] is None or not defined), you may have trouble when calling `.render()`.
  logger.warn(


Max X: 594
Max World: 1
Max Stage: 1


![image](images/random_agent.gif)

You can see that the random agent never gets past the second pipe. This is because it is not probabilistically reasonable for random inputs to know to sustain a jump by pressing and holding A to get high enough to clear the pipe and keep going. This pipe exists at an X value of 595/596. Let's see if there are any other agents that can get farther.

<a id='heuristic-agent'></a>

## Heuristic Agent

Although it is not reinforcement learning, I decided to implement a basic heuristic model that uses a simple algorithm to try to beat a level of Super Mario Bros which will serve as our milestone to beat. If our RL agent is able to surpass the heuristic agent, we will consider the RL agent to be a success.

In [37]:
# Create the Super Mario Bros. environment
env = gym.make('SuperMarioBros-v0')

# Set the Joypad wrapper
env = JoypadSpace(env.env, SIMPLE_MOVEMENT)

# Define a new reset function to accept `seeds` and `options` args for compatibility
def gymnasium_reset(self, **kwargs):
    return self.env.reset(), {}

# Overwrite the old reset to accept `seeds` and `options` args
env.reset = gymnasium_reset.__get__(env, JoypadSpace)

# Set TimeLimit back
env = TimeLimit(StepAPICompatibility(env, output_truncation_bool=True), max_episode_steps=steps)

# create global variables for inputs
done = True
going_up = False
prev_y = None
min_y = 0
max_y = 0

max_x = 0
max_world = 0
max_stage = 0

for step in range(1700):
    if done:
        state = env.reset()
        prev_y = None
        hold_jump = False
    
    # if Mario is on flat groun
    # or in the process of rising from previous jump
    # will continue to hold A to perform the maximum jump
    action = SIMPLE_MOVEMENT.index(['right', 'A', 'B']) if going_up else SIMPLE_MOVEMENT.index(['right', 'B'])
    state, reward, done, truncated, info = env.step(action)

    # set going_up to true if Mario is not descending
    if prev_y is not None:
        if info['y_pos'] >= prev_y:
            going_up = True
        else:
            going_up = False

    # capture current y position to compare for next state
    prev_y = info['y_pos']
        
    if info['world'] > max_world:
        max_world = info['world']
        max_x = 0
        max_stage = 1
    elif info['stage'] > max_stage:
        max_stage = info['stage']
        max_x = 0
    elif info['world'] == max_world and info['stage'] == max_stage and info['x_pos'] > max_x:
        max_x = info['x_pos']

    if done or truncated:
        done = True
    env.render()
    time.sleep(0.01)  # Add a delay of 0.01 seconds between frames to slow down render

# Close the environment
env.close()
print(f"Max X: {max_x}")
print(f"Max World: {max_world}")
print(f"Max Stage: {max_stage}")

KeyboardInterrupt: 

![image](images/heuristic_agent.gif)

This heuristic, while not an actual learning model, is effective by sheer luck. This strategy is similar to what an actual player might try when attempting to beat the first level. It is intuitive to try and jump as high and as often as possible to clear most obstacles and enemies. It is by sheer luck (and some deaths) that the heuristic is able to avoid enemies and pits. Despite this, however, this agent is able to clear the first level, although it does not get very far in the second level.

Let's try to create a reinforcement learning model and see if the agent can beat the level without relying on luck.

<a id='ppo-agent'></a>


## PPO Agent


Let's actually have an agent that learns, rather than blindly jumping. The initial model of choice will be a Proximal Policy Optimization (PPO) agent. This type of reinforcement learning model learns a policy that directly maps an approximation of the optimal action for any given state to exploit  to approximate the optimal action for any given state to maximize the reward returned over time.

It does this by using gradient ascent to maximize the policy using the clipped objective function, which helps avoid making drastic weight updates:

$L(\theta) = \mathbb{E} \left[ \min \left( \frac{\pi_{\theta}(a|s)}{\pi_{\theta_{\text{old}}}(a|s)} \hat{A}, \text{clip}\left(\frac{\pi_{\theta}(a|s)}{\pi_{\theta_{\text{old}}}(a|s)}, 1 - \epsilon, 1 + \epsilon \right) \hat{A} \right) \right]$

This function seeks to optimize the parameters ($\theta$) of the neural network by evaluating the expected value ($\mathbb{E}$) of the minimum between the following two values:

$$\frac{\pi_{\theta}(a|s)}{\pi_{\theta_{\text{old}}}(a|s)} \hat{A}$$

<p style="text-align: center;">or</p>

$$\text{clip}\left(\frac{\pi_{\theta}(a|s)}{\pi_{\theta_{\text{old}}}(a|s)}, 1 - \epsilon, 1 + \epsilon \right) \hat{A}$$

$\frac{\pi_{\theta}(a|s)}{\pi_{\theta_{\text{old}}}(a|s)}$ is the policy ratio, or how much the new policy $\pi_\theta$ has changed in relation to the old policy $\pi_{old}$ and $\hat A$ is the advantage, or how much better or worse the action $a$ is compared to the average action taken in state $s$

The first term represents the weight change when the policy ratio is directly multiplied by the advantage. The second uses the clipping function to constrain the value of the policy ratio within the range $1 - \epsilon$ and $1 + \epsilon$ before multiplying with the advantage $\hat A$

Finding the expected value of the minimum of these two terms results in a gradient ascent that should prevent drastic policy updates and ensure more stable learning.

In [34]:
# Train the agent

import numpy as np
import gym_super_mario_bros
import time
import gym # v0.26.2
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
from gym.wrappers import GrayScaleObservation
from gym.wrappers import StepAPICompatibility, TimeLimit
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack, VecMonitor
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback, CheckpointCallback
from gym.wrappers import GrayScaleObservation, ResizeObservation, FrameStack
from gym import Wrapper
from gym import RewardWrapper
import os

class SkipFrame(Wrapper):
    def __init__(self, env, skip):
        super().__init__(env)
        self.skip = skip
    
    def step(self, action):
        # create a reward accumulator
        reward_accum = 0.0
        done = False
        for _ in range(self.skip):
            next_state, reward, done, truncated, info = self.env.step(action)
            # add reward to accumulator
            reward_accum += reward
            if done:
                break
        return next_state, reward_accum, done, truncated, info

# final custom reward calculation after failed training (see below for details)
class CustStepReward(Wrapper):
    def __init__(self, env):
        super().__init__(env)
        self.prev_y_pos = 0
        self.reward_mean = 0
        self.reward_var = 1
        self.num_rewards = 0
        self.succesive_A_presses = 0

    def step(self, action):
        done = False
        next_state, reward, done, truncated, info = self.env.step(action)
        # if rising, agent must have pressed A state before
        if self.prev_y_pos < info['y_pos']:
            self.succesive_A_presses += 1
            reward += 2 * self.succesive_A_presses # incentivize jumping higher if already jumping
        # agent is not rising
        else:
            # reset to 0
            # no reward (and no penalty)
            self.succesive_A_presses = 0
        self.prev_y_pos = info['y_pos']
        
        if info['life'] < 2:
            reward -= 50 # heavily penalize death and end the episode
            done = True
        if info['flag_get']:
            reward += 100 # heavily incentivize beating the level
        
        # Update reward statistics
        self.num_rewards += 1
        old_mean = self.reward_mean
        self.reward_mean += (reward - self.reward_mean) / self.num_rewards
        self.reward_var += (reward - old_mean) * (reward - self.reward_mean)
        
        # Normalize reward
        reward_std = np.sqrt(self.reward_var / self.num_rewards)
        normalized_reward = (reward - self.reward_mean) / (reward_std + 1e-8)
        return next_state, normalized_reward, done, truncated, info


# Create the Super Mario Bros. environment
env = gym.make('SuperMarioBros-v0')
steps = env._max_episode_steps

CUSTOM_ACTIONS = [
    ['right', 'A', 'B'],
    ['right','B']
]

# Set the Joypad wrapper
env = JoypadSpace(env.env, CUSTOM_ACTIONS)
# Overwrite the old reset to accept seeds and options args
def gymnasium_reset(self, **kwargs):
    return self.env.reset(), {}
env.reset = gymnasium_reset.__get__(env, JoypadSpace)

env = StepAPICompatibility(env, output_truncation_bool=True)
#env = CustStepReward(env)

#env = SkipFrame(env, skip=4)
#env = ResizeObservation(env, shape=84) # reduce size of frame image
#env = GrayScaleObservation(env) # create grayscale images
#env = FrameStack(env, num_stack=8, lz4_compress=True) # stack frames

model = PPO(
    'CnnPolicy',      # Use a convolutional neural network
    env,              # Game environment
    verbose=1,        # print diagnostics
    learning_rate=3e-4,  # how much to adjust the model with each step
    n_steps=512,      # frequency of updates
    batch_size=128,   # number of state samples evaluated in clipping objective function
    clip_range = 0.2, # range to clip policy ratio in the clipping objective function
    ent_coef = 0.9,   # entropy coefficient to encourage exploration
    gamma = 0.99      # diminishes rewards for future action-state returns
)


# uncomment if continuing a previous training session
#model.load(os.path.join("trained_agents","ppo","ppo_mario.zip"))

# Define evaluation and checkpoint callbacks
eval_callback = EvalCallback(env, best_model_save_path=os.path.join('trained_agents','ppo'), log_path=os.path.join('trained_agents','ppo','logs'), eval_freq=5000, deterministic=True, render=False)
checkpoint_callback = CheckpointCallback(save_freq=25000, save_path=os.path.join('trained_agents','ppo'), name_prefix='ppo_mario')

model.learn(total_timesteps=700000, callback=[eval_callback, checkpoint_callback])

model.save(os.path.join('trained_agents','ppo','ppo_mario_2'))

# Evaluate the agent
print("evaluating policy...")
mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=10)
env.close()
print(f"Mean reward: {mean_reward} ± {std_reward}")


c:\Users\sethe\AppData\Local\Programs\Python\Python312\Lib\site-packages\gym\envs\registration.py:555: UserWarning: WARN: The environment SuperMarioBros-v0 is out of date. You should consider upgrading to version `v3`.
  logger.warn(


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


c:\Users\sethe\AppData\Local\Programs\Python\Python312\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
c:\Users\sethe\AppData\Local\Programs\Python\Python312\Lib\site-packages\gym\utils\passive_env_checker.py:195: UserWarning: WARN: The result returned by `env.reset()` was not a tuple of the form `(obs, info)`, where `obs` is a observation and `info` is a dictionary containing additional information. Actual type: `<class 'numpy.ndarray'>`
  logger.warn(
c:\Users\sethe\AppData\Local\Programs\Python\Python312\Lib\site-packages\stable_baselines3\common\callbacks.py:414: UserWarning: Training and eval env are not of the same type<stable_baselines3.common.vec_env.vec_transpose.VecTransposeImage object at 0x000002E952

----------------------------
| time/              |     |
|    fps             | 128 |
|    iterations      | 1   |
|    time_elapsed    | 3   |
|    total_timesteps | 512 |
----------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 82         |
|    iterations           | 2          |
|    time_elapsed         | 12         |
|    total_timesteps      | 1024       |
| train/                  |            |
|    approx_kl            | 0.02697697 |
|    clip_fraction        | 0.173      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.68      |
|    explained_variance   | 0.00227    |
|    learning_rate        | 0.0003     |
|    loss                 | 1.82       |
|    n_updates            | 10         |
|    policy_gradient_loss | 0.0105     |
|    value_loss           | 132        |
----------------------------------------
-----------------------------------------
| time/          

c:\Users\sethe\AppData\Local\Programs\Python\Python312\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=5000, episode_reward=741.00 +/- 0.00
Episode length: 319.00 +/- 0.00
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 319         |
|    mean_reward          | 741         |
| time/                   |             |
|    total_timesteps      | 5000        |
| train/                  |             |
|    approx_kl            | 0.017280929 |
|    clip_fraction        | 0.0418      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.688      |
|    explained_variance   | 0.165       |
|    learning_rate        | 0.0003      |
|    loss                 | 13.1        |
|    n_updates            | 90          |
|    policy_gradient_loss | -0.00737    |
|    value_loss           | 48.5        |
-----------------------------------------
New best mean reward!
-----------------------------
| time/              |      |
|    fps             | 60   |
|    iterations      | 10   |
|    time_elap

KeyboardInterrupt: 

To optimize the learning process, I decided to use several wrappers for optimization. The idea for these optimizations came from a video by Sourish Kundu I used as the basis for the Q-Learning model implemented later in this project (Kundu, 2023). The following additions implemented above along with configuring Stable Baselines to use the GPU sped up the learning process quite a bit and allowed me to train the model for 400,000 episodes over the course of only a few hours.

The first optimization is to create a custom wrapper that skips every four frames. Since the NES Mario Bros runs at about 60 fps, not much changes in the game state for each frame and the agent will choose an action and repeat the action for every four frames.

Additional optimizations included downgrading from color RGB pixel values to gray scale values, reducing the image size, and stacking 8 frames at a time. The framestacking, when combined with the frame skipping, reduces the processing complexity of the model and at the same time allow it to interpret more meaningful game state changes than it would otherwise evaluate from every individual frame.

The actual architecture of the model uses a convolutional neural network __use model.policy to determine model architecture__.

In [29]:
# Test the trained agent

import numpy as np
import gym_super_mario_bros
import time
import gym
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
from gym.wrappers import GrayScaleObservation
from gym.wrappers import StepAPICompatibility, TimeLimit
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack, VecMonitor
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback, CheckpointCallback
import os

# Create the Super Mario Bros. environment
env = gym.make('SuperMarioBros-v0')
steps = env._max_episode_steps
CUSTOM_ACTIONS = [
    ['right', 'A', 'B'],
    ['right','B']
]
# Set the Joypad wrapper
env = JoypadSpace(env.env, SIMPLE_MOVEMENT)
# Overwrite the old reset to accept seeds and options args
def gymnasium_reset(self, **kwargs):
    return self.env.reset(), {}
env.reset = gymnasium_reset.__get__(env, JoypadSpace)

env = StepAPICompatibility(env, output_truncation_bool=True)
#env = CustStepReward(env)

#env = SkipFrame(env, skip=4)
#env = ResizeObservation(env, shape=84) # reduce size of frame image
#env = GrayScaleObservation(env) # create grayscale images
#env = FrameStack(env, num_stack=8, lz4_compress=True) # stack frames

# uncomment to load saved trained model
# model = PPO.load(os.path.join("trained_agents","ppo","ppo_mario_25000_steps.zip"))

max_x = 0
max_world = 0
max_stage = 0

obs, info = env.reset()
for step in range(2000):
    action, info = model.predict(np.array(obs))
    action = action.item()       
    obs, reward, done, truncated, info = env.step(action)
    if info['world'] > max_world:
        max_world = info['world']
        max_x = 0
        max_stage = 1
    elif info['stage'] > max_stage:
        max_stage = info['stage']
        max_x = 0
    elif info['world'] == max_world and info['stage'] == max_stage and info['x_pos'] > max_x:
        max_x = info['x_pos']
    env.render()
    time.sleep(0.05)
    if done:
        obs, info = env.reset()

# Close the environment
env.close()
print(f"Max X: {max_x}")
print(f"Max World: {max_world}")
print(f"Max Stage: {max_stage}")

KeyboardInterrupt: 

![image](images/initial_ppo.gif)

With the default PPO model values, the agent is unable to clear the pipe at the 595 X coordinate. I allowed the agent to train for 100,000 episodes hoping it would figure out that keeping the A button held down would lead to a higher jump, allowing it to continue to the right. Unfortunately, after 100,000 iterations, the model instead runs directly into the first goomba over and over.

![image](images/goomba_death.gif)

I attempted to fine tune the learning rate using a scheduler, fine tuning the gamma and clip values, as well as included an entropy coefficient to encourage exploration to no avail. Even with a large entropy coefficient, the model was not able to get over the pipe by chance. In retrospect, this is consistent with the behavior with our random agent, as the chances of stringing together several A button presses while also holding right for forward momentum could be rather unlikely. Even with the addition of frame skipping, which causes the agent to perform an action for every four frames, the agent was unable to clear the pipe in any of the learning episodes.

__Because of this, the agent always gets stuck at either the first or second pipe and accrues penalties for the clock ticking down with no forward progress, per the default reward structure. After 100,000 episodes, the agent seems to learn that in order to avoid this penalty, it is more optimal to run directly into the first goomba, receiving a smaller penalty from taking a death than the accrued penalty it receives from getting stuck at the pipe.__

To counteract this, I added a new Gym environment wrapper that returns a new reward function. Learning from the heuristic agent, I believe Mario will both progress farther and avoid death more if he spends more time jumping. Because of this, I added a reward if Mario's change in Y-value is positive (meaning he is ascending). This should hopefully incentivize the agent to continuously press the A button and increase the chances that it is able to clear any tall pipes.

Additionally, I chose to give a heavy penalty and terminate the episode when the agent dies, which should discourage the agent from exploiting any rewards it receives from taking an intentional death.

Lastly, I added an incentive for the agent if the 'get_flag' bool is true, further incentivizing the agent to complete the level.

Lastly, I normalized the returned reward based on the average reward of the episode in order to stabilize the learning process while I made adjustments:

```python
class CustStepReward(Wrapper):
    def __init__(self, env):
        super().__init__(env)
        self.prev_y_pos = 0
        self.reward_mean = 0
        self.reward_var = 1
        self.num_rewards = 0

    def step(self, action):
        done = False
        next_state, reward, done, truncated, info = self.env.step(action)
        # create custom reward value
        if self.prev_y_pos < info['y_pos']:
            reward += 4 # incentivize being airborn
        self.prev_y_pos = info['y_pos']
        if info['life'] < 2:
            reward -= 50 # heavily penalize death and end the episode
            done = True
        if info['flag_get']:
            reward += 100 # heavily incentivize beating the level
        
        # Update reward statistics
        self.num_rewards += 1
        old_mean = self.reward_mean
        self.reward_mean += (reward - self.reward_mean) / self.num_rewards
        self.reward_var += (reward - old_mean) * (reward - self.reward_mean)
        
        # Normalize reward
        reward_std = np.sqrt(self.reward_var / self.num_rewards)
        normalized_reward = (reward - self.reward_mean) / (reward_std + 1e-8)
        return next_state, normalized_reward, done, truncated, info
```

The end result worked as expected, and the agent was able to get farther in the level after training for 100,000 episodes. __However, after training for 400,000 episodes, the model again resorted to exploitation, this time using the reward for increasing y position. The reward it received for this outweighed the penalty for staying still and it found that the optimal policy was to stay in one place and jump continuously over and over to accrue rewards until the time ran out.__

![image](images/left_model.gif)

To fix this, I revised the reward function again to only reward increasing y position if already in a jumping state and scale this reward by the number of successive A presses. This should hopefully encourage the agent to jump higher to gain a higher reward:

```python
if self.prev_y_pos < info['y_pos']:
    self.succesive_A_presses += 1
    reward += 2 * self.succesive_A_presses # incentivize jumping higher if already jumping
# agent is not rising
else:
    # reset to 0
    # no reward (and no penalty)
    self.succesive_A_presses = 0
self.prev_y_pos = info['y_pos']
```

Additionally, I simplified the movement actions to only `Right + B` for running right, and `Right + B + A` for a running jump.

```python
CUSTOM_ACTIONS = [
    ['right', 'A', 'B'],
    ['right','B']
]
```

While this simplification may make it harder for the agent to avoid enemies, it is more in alignment with the heuristic agent we used as a baseline and should hopefully allow the agent to make better progress.

With these revisions, the agent was able to complete the level and even get a bit farther after 600,000 training steps.

![image](images/ppo_final.gif)

This reward function seems to work as intedned and the model learns with quite a bit of stability. Given enough iterations, I believe the model would continue to improve and progress farther into the game.



<a id='q-learning-agent'></a>

## Q-Learning Agent

To compare with the Stable Baseling3 PPO agent, I also create a Q-Learning agent. As referenced before, a huge thank you to Sourish Kundu, who made a video on how to do exactly this with some great explanations (Kundu, 2023). If you have a chance, I highly recommend giving it a watch, even if you are not working with this gym environment, as it gives some great explanations of general concepts we have been discussing in class.

https://www.youtube.com/watch?v=_gmQZToTMac

https://github.com/Sourish07/Super-Mario-Bros-RL

We can begin by creating the same wrappers as before. Initially, I used these wrappers for the Q-Learning model and applied them to the PPO model after the fact. We again skip 4 frames, rescale and reduce the images to gray scale while stacking 4 frames. I am also including the previous reward wrapper used in the PPO model before applying all these wrappers together in the function `apply_wrappers` that returns the environment utilizing all of the environment wrappers.

One note is that the following algorithm requires gym version 0.24.0 as the returns of the reset function need to be a single value, as opposed to the tuple returned in later versions. Overriding the reset function does not work, as several of the necessary wraooers and Python modules are not compatible with the newer versions of gym. If you are attempting to run this cell, you will receive an error unless you run `pip install gym==0.24.0` first.

In [1]:
import numpy as np
from gym import Wrapper
from gym.wrappers import GrayScaleObservation, ResizeObservation, FrameStack

class SkipFrame(Wrapper):
    def __init__(self, env, skip):
        super().__init__(env)
        self.skip = skip
    
    def step(self, action):
        # create a reward accumulator
        reward_accum = 0.0
        done = False
        for _ in range(self.skip):
            next_state, reward, done, info = self.env.step(action)
            # add reward to accumulator
            reward_accum += reward
            if done:
                break
        return next_state, reward_accum, done, info
    
class CustStepReward(Wrapper):
    def __init__(self, env):
        super().__init__(env)
        self.prev_y_pos = 0
        self.reward_mean = 0
        self.reward_var = 1
        self.num_rewards = 0
        self.succesive_A_presses = 0

    def step(self, action):
        done = False
        next_state, reward, done, info = self.env.step(action)
        # if rising, agent must have pressed A state before
        if self.prev_y_pos < info['y_pos']:
            self.succesive_A_presses += 1
            reward += 2 * self.succesive_A_presses # incentivize jumping higher if already jumping
        # agent is not rising
        else:
            # reset to 0
            # no reward (and no penalty)
            self.succesive_A_presses = 0
        self.prev_y_pos = info['y_pos']
        
        if info['life'] < 2:
            reward -= 50 # heavily penalize death and end the episode
            done = True
        if info['flag_get']:
            reward += 100 # heavily incentivize beating the level
        
        # Update reward statistics
        self.num_rewards += 1
        old_mean = self.reward_mean
        self.reward_mean += (reward - self.reward_mean) / self.num_rewards
        self.reward_var += (reward - old_mean) * (reward - self.reward_mean)
        
        # Normalize reward
        reward_std = np.sqrt(self.reward_var / self.num_rewards)
        normalized_reward = float((reward - self.reward_mean) / (reward_std + 1e-8))
        return next_state, normalized_reward, done, info

def apply_wrappers(env):
    env = SkipFrame(env, skip=4) # skip every four frames
    env = ResizeObservation(env, shape=84) # reduce size of frame image
    env = GrayScaleObservation(env) # create grayscale images
    env = FrameStack(env, num_stack=4, lz4_compress=True) # stack frames (4 skipped)
    env = CustStepReward(env)
    return env

Instead of using a default neural network, we will create our own using convolution layers and linear layers. We are using Conv2d layers with Relu activation functions, as these are typically appropriate for images (as I learned from HW5). Using MaxPool2d will reduce spatial dimensions and lower computational cost, which will hopefully lower the processing time for our algorithm.

Additional methods are used to dynamically evaluate the input shape for our initial linear layer (`_get_conv_out`), prevent pytorch from updating the gradients if frozen (`_freeze`), and tell pytorch how to handle the forward pass for each tensor (`forward`).

I reduced the complexity of the architecture from Sourish's video to improve processing speed, but much of the structure of the attributes and methods were influenced heavily by his example.

In [2]:
import torch
from torch import nn
import numpy as np

class AgentNN(nn.Module):
    def __init__(self, input_shape, n_actions, freeze=False):
        super().__init__()
        # Conolutional layers
        self.conv_layers = nn.Sequential(
            nn.Conv2d(input_shape[0], 32, kernel_size=5, stride=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(32, 64, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        # use built-in method to get the dimensional input size for initial linear layer
        conv_out_size = self._get_conv_out(input_shape)

        # Fully connected linear layers
        self.network = nn.Sequential(
            self.conv_layers,
            nn.Flatten(),
            nn.Linear(conv_out_size, 256),
            nn.ReLU(),
            nn.Linear(256, n_actions) # determine best action to predict
        )

        # call the freeze method if frozen
        # to make sure no parameters are updated if frozen
        if freeze:
            self._freeze()
        
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu' # try to use the GPU if possible
        self.to(self.device)

    # method to handle forward pass 
    def forward(self, x):
        # pass the input tensor through the neural network layers
        return self.network(x)

    # get the number of neurons for our linear layers
    def _get_conv_out(self, shape):
        o = self.conv_layers(torch.zeros(1, *shape))
        return int(np.prod(o.size()))
    
    # method to make sure gradients are not calculated if frozen
    def _freeze(self):        
        for p in self.network.parameters():
            p.requires_grad = False

Next, we will create our agent class to use the neural network created previously:

In [3]:
import torch
import numpy as np
from tensordict import TensorDict # use tensors in python lists to speed up processing
from torchrl.data import TensorDictReplayBuffer, LazyMemmapStorage
import psutil
import os

class Agent:
    def __init__(self, 
                 input_dims, 
                 num_actions, 
                 lr=0.00025, 
                 gamma=0.9, 
                 epsilon=1.0, 
                 eps_decay=0.99999975, 
                 eps_min=0.1, 
                 replay_buffer_capacity=150000, 
                 batch_size=32, 
                 sync_network_rate=10000
                 ):
        
        self.num_actions = num_actions # use the appropriate number of actions (SIMPLE_MOVEMENT dict has 7)
        self.learn_step_counter = 0

        # Hyperparameters
        self.lr = lr
        self.gamma = gamma
        self.epsilon = epsilon
        self.eps_decay = eps_decay
        self.eps_min = eps_min
        self.batch_size = batch_size
        self.sync_network_rate = sync_network_rate

        # Networks
        self.online_network = AgentNN(input_dims, num_actions)
        self.target_network = AgentNN(input_dims, num_actions, freeze=True)

        # Optimizer and loss
        self.optimizer = torch.optim.Adam(self.online_network.parameters(), lr=self.lr)
        self.loss = torch.nn.MSELoss() # loss function

        # Replay buffer
        storage = LazyMemmapStorage(replay_buffer_capacity)
        self.replay_buffer = TensorDictReplayBuffer(storage=storage)
        self.log_memory_usage()

    def log_memory_usage(self):
        process = psutil.Process(os.getpid())
        mem_info = process.memory_info()
        print(f"Memory Usage: {mem_info.rss / 1024 ** 2:.2f} MB")

    def choose_action(self, observation):
        # create the potential to choose a random action
        # this will include some value of randomness to increase exploration
        if np.random.random() < self.epsilon:
            return np.random.randint(self.num_actions)
        
        observation = (
            torch.tensor(np.array(observation), dtype=torch.float32) # speed up processing by using tensors instead of numpy arrays
            .unsqueeze(0) # add dimension of batch size to first index of tensor
            .to(self.online_network.device) # move to the correct device (GPU or CPU)
        )
        # return the action with the highest Q-value
        return self.online_network(observation).argmax().item()
    
    # compute the value of epsilon to diminish rewards for later actions
    def decay_epsilon(self):
        self.epsilon = max(self.epsilon * self.eps_decay, self.eps_min)

    # put tensors in a dict and add to buffer
    def store_in_memory(self, state, action, reward, next_state, done):
        # Create TensorDict with correct shapes and types
        data = TensorDict({
            "state": torch.tensor(np.array(state), dtype=torch.float32),
            "action": torch.tensor(action),
            "reward": torch.tensor(reward),
            "next_state": torch.tensor(np.array(next_state), dtype=torch.float32),
            "done": torch.tensor(done)
        }, batch_size=[])
        self.replay_buffer.add(data)
    
    # copy weights of online network to target network if enough steps have passed
    def sync_networks(self):
        if self.learn_step_counter % self.sync_network_rate == 0 and self.learn_step_counter > 0:
            self.target_network.load_state_dict(self.online_network.state_dict())

    # save current model (in case something goes wrong)
    def save_model(self, path):
        torch.save(self.online_network.state_dict(), path)

    # load model
    def load_model(self, path):
        self.online_network.load_state_dict(torch.load(path))
        self.target_network.load_state_dict(torch.load(path))


    def learn(self):
        # if not enough experiences, return and keep going
        if len(self.replay_buffer) < self.batch_size:
            return
        
        # copy weights to target network
        self.sync_networks()
        
        # clear gradients
        self.optimizer.zero_grad()

        # sample the replay buffer and store the results
        samples = self.replay_buffer.sample(self.batch_size).to(self.online_network.device)
        states = samples['state']
        actions = samples['action']
        rewards = samples['reward']
        next_states = samples['next_state']
        dones = samples['done']

        # get the predicted values from our neural network with the appropriate batch size
        predicted_q_values = self.online_network(states)
        predicted_q_values = predicted_q_values[np.arange(self.batch_size), actions.squeeze()]

        # Max returns two tensors, the first one is the maximum value, the second one is the index of the maximum value
        target_q_values = self.target_network(next_states).max(dim=1)[0]
        # The rewards of any future states don't matter if the current state is a terminal state
        # If done is true, then 1 - done is 0, so the part after the plus sign (representing the future rewards) is 0
        target_q_values = rewards + self.gamma * target_q_values * (1 - dones.float())

        loss = self.loss(predicted_q_values, target_q_values)
        loss.backward()
        self.optimizer.step()

        self.learn_step_counter += 1
        self.decay_epsilon()

This class performs an approximate mathematical implementation of the Bellman Equation for calculating Q-values for each state-action pair:

$Q(s, a) \leftarrow Q(s, a) + \alpha \left[ r + \gamma \max_{a'} Q(s', a') - Q(s, a) \right]$

As seen here:

```python
target_q_values = rewards + self.gamma * target_q_values * (1 - dones.float())
```

PyTorch's built-in Mean Squared Error loss function is applied between predicted and target Q-values:
```python
self.losss = torch.nn.MSE()
```
```python
loss = self.loss(predicted_q_values, target_q_values)
```

This is then minimized using backpropagation in
```python
loss.backward(); self.optimizer.step() in learn())`.
```

The agent selects actions using an epsilon-greedy policy  as seen in the `choose_action()` function, where the epsilon value represents the probability the model chooses a random action, selecting the action with the highest Q-value the rest of the time. This epsilon value decays overtime as seen here:

```python
self.epsilon = max(self.epsilon * self.eps_decay, self.eps_min`
```

To stabilize learning, the agent periodically synchronizes the target network with the online network `self.sync_networks()`, copying the weights from the online network to the target network after a certain number of steps.

We now have everything we need to create the environment and run our reinforcement learning architecture:

In [5]:
# Train the model
import gym_super_mario_bros
from gym_super_mario_bros.actions import RIGHT_ONLY
from nes_py.wrappers import JoypadSpace
import numpy as np
import gym
import gym_super_mario_bros
import time
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
import gym_super_mario_bros
from gym_super_mario_bros.actions import RIGHT_ONLY
from nes_py.wrappers import JoypadSpace

ENV_NAME = 'SuperMarioBros-1-1-v0'
SAVE_INTERVAL = 1000
DISPLAY = True
NUM_OF_EPISODES = 100000

CUSTOM_ACTIONS = [
    ['right','A','B'],
    ['right','B']
]

env = gym_super_mario_bros.make(ENV_NAME)
env = JoypadSpace(env, CUSTOM_ACTIONS)
env = apply_wrappers(env)

agent = Agent(input_dims=env.observation_space.shape, num_actions=env.action_space.n)

# create a progress bar when training
def print_progress(cur,end, bar_length=40):
    progress = cur / end
    block = int(round(bar_length * progress))
    text = f"\rCurrently processing episode {cur}/{end} [{'#' * block + '-' * (bar_length - block)}] {progress * 100:.2f}%"
    print(text, end='', flush=True)

for i in range(NUM_OF_EPISODES):
    print_progress(i+1,NUM_OF_EPISODES)
    done = False
    state = env.reset()
    while not done:
        a = agent.choose_action(state)
        new_state, reward, done, info  = env.step(a)
        
        agent.store_in_memory(state, a, reward, new_state, done)
        agent.learn()

        state = new_state

        # save the model every save interval
        if (i + 1) % SAVE_INTERVAL == 0:
            agent.save_model(os.path.join("trained_agents\q_learning",str(i + 1) + "_q_agent.pt"))

env.close()

<>:52: SyntaxWarning: invalid escape sequence '\q'
<>:52: SyntaxWarning: invalid escape sequence '\q'
C:\Users\sethe\AppData\Local\Temp\ipykernel_33160\3476472889.py:52: SyntaxWarning: invalid escape sequence '\q'
  agent.save_model(os.path.join("trained_agents\q_learning",str(i + 1) + "_q_agent.pt"))


Memory Usage: 383.50 MB
Currently processing episode 2/5 [################------------------------] 40.00%

C:\Users\sethe\AppData\Local\Temp\ipykernel_33160\3476472889.py:52: SyntaxWarning: invalid escape sequence '\q'
  agent.save_model(os.path.join("trained_agents\q_learning",str(i + 1) + "_q_agent.pt"))


KeyboardInterrupt: 

In [4]:
# Test the model
import gym_super_mario_bros
from gym_super_mario_bros.actions import RIGHT_ONLY
from nes_py.wrappers import JoypadSpace
import numpy as np
import gym
import gym_super_mario_bros
import time
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
import gym_super_mario_bros
from gym_super_mario_bros.actions import RIGHT_ONLY
from nes_py.wrappers import JoypadSpace

ENV_NAME = 'SuperMarioBros-1-1-v0'
SHOULD_TRAIN = True
DISPLAY = True
NUM_OF_EPISODES = 5

CUSTOM_ACTIONS = [
    ['right', 'A', 'B'],
    ['right','B']
]

env = gym_super_mario_bros.make(ENV_NAME)
env = JoypadSpace(env, CUSTOM_ACTIONS)
env = apply_wrappers(env)

agent = Agent(input_dims=env.observation_space.shape, num_actions=env.action_space.n)

agent.load_model(os.path.join("trained_agents","q_learning","8000_q_agent.pt"))

max_x = 0
max_world = 0
max_stage = 0

state = env.reset()
for step in range(1500):
    action = agent.choose_action(state)
    state, reward, done, info = env.step(action)
    if info['world'] > max_world:
        max_world = info['world']
        max_x = 0
        max_stage = 1
    elif info['stage'] > max_stage:
        max_stage = info['stage']
        max_x = 0
    elif info['world'] == max_world and info['stage'] == max_stage and info['x_pos'] > max_x:
        max_x = info['x_pos']
    env.render()
    time.sleep(0.02)
    if done:
        state = env.reset()

print(f"Max X: {max_x}")
print(f"Max World: {max_world}")
print(f"Max Stage: {max_stage}")

env.close()

c:\Users\sethe\AppData\Local\Programs\Python\Python312\Lib\site-packages\gym\envs\registration.py:568: UserWarning: WARN: The environment SuperMarioBros-1-1-v0 is out of date. You should consider upgrading to version `v3`.
  logger.warn(
c:\Users\sethe\AppData\Local\Programs\Python\Python312\Lib\site-packages\gym\envs\registration.py:619: UserWarning: WARN: Env check failed with the following message: Calling the reset method with `return_info=True` did not return a 2-tuple
You can set `disable_env_checker=True` to disable this check.
  logger.warn(


Memory Usage: 612.57 MB


C:\Users\sethe\AppData\Local\Temp\ipykernel_20728\3851061806.py:93: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.online_network.load_state_dict(torch.load(path))
C:\Us

Max X: 1433
Max World: 1
Max Stage: 1


![image](images/q_learning_4000.gif)

WIth only 4,000 steps, the Q Learning algorithm is able to take advantage of the height reward and is clearing the tall pipes. Given enough iterations, I believe this algorithm would show similar, if not better results than our PPO algorithm. However, it took the same amount of time for me to run this algorithm for 4,000 iterations as it did for me to run the PPO algorithm for 600,000 steps. It is interesting to note the immediacy of the rewards when the PPO agent was still unable to clear any pipes at the 4,000th iteration.

<a id='results'></a>

## Results


*show the result and interpretation of your experiment. Any iterative improvements summary.*

5. Does it include the results summary, interpretation of experiments and visualization (e.g. performance comparison table, graphs etc)? (7)


| Model     | Video of Model Predictions  | Notes    |Max World       | Max Level | Max X Position | Number of Training Steps  |
|------------------|---- |---------------|----------------|-----------|-----------|-----------|
| __Random Agent__     |  ![image](images/random_agent.gif)   |    Environment is working, but agent cannot progress far with random actions.        | 1            | 1         | 595         | N/A | 
| __Heuristic Agent__  | ![image](images/heuristic_agent.gif)   |  Agent wins using strategy and luck (not reinforcement learning)         | 1            | 2         | 199         | N/A |
| __PPO Agent__        |  ![image](images/ppo_final.gif)   | Refining reward function keeps the agent from exploiting default rewards with unintended behavior        | 1            | 2         | 482         | 600,000 |
| __Q Learning Agent__ | ![images](images/q_learning_4000.gif)   | Model is stable, but very slow         | 1            | 1         | 1423         | 5,000 |

<a id='conclusion'></a>

## Conclusion


*Conclusion, discussion, reflection, or suggestions for future improvements or future ideas.*

6. Does it include discussion (what went well or not and why), and suggestions for improvements or future work? (5)

In the end the heuristic created a strong strategy for simplifying the model's action choices, and creating a set of custom actions to either run right, or run and jump right encouraged the model to control Mario similar to a real player, jumping over most obstacles and minimizing the chances for death by running into enemies on the ground.

Utilizing the custom reward function to encourage higher jumping and creating the simplified action list allowed both the PPO and Q-Learning agents to progress much farther than initially, and ensured stable learning that balanced both exploration and exploitation. Prior to these revisions, the model continuously exploited the reward mechanics to a point that it was not progressing further into the game and would eventually converge on a policy that either stayed in one position (either stuck at a pipe or jumping over and over) or killing itself to the firts goomba repeatedly.

The main difference in performance between the Q-Learning and PPO models was mainly down to processing complexity. The PPO model was simpler and was able to perform hundreds of thousands of training episodes in only a few hours, whereas the Q-Learning algorithm, while it was significantly more stable, took over 24 hours just to get to 50,000 training episodes. The benefits of the processing speed of Stable Baseline3's PPO library enabled me to see progress quickly and make adjustments as needed, whereas it was relatively difficult to figure out any problems with the Q-Learning algorithm until a considerable amount of time was wasted.

Because of the processing drawbacks, I was forced to consider further optimizations to improve performance which I could use to speed up the SB3 PPO model even more.

At the same time, I was able to implement the improvements made to the reward and action choices of the PPO model to further improve the Q-Learning model. Insights which would have taken days and weeks to gather if I simply used the Q-Learning agent alone.

It would be interesting to see how much the Q-Learning agent would improve given the same number of iterations the PPO model was able to make and compare them side by side. Additionally, I have seen other projects using this model that created custom wrappers that are able to read the memory of the Super Mario Bros environment to be able to have even more information for the agent to utilize, effectively being able to determine locations of enemies, blocks, pits, etc.

Given additional time, I like to see how well the agent would be able to improve with this information. This in combination with using random stage selection could also allow the model to be able to create a more generalized optimal policy that could assist in beating many levels of the game. As it is, my model seems to be making progress very slowly as it learns from trial and error as opposed to dynamically "seeing" information in the game state and choosing actions accordingly.

<a id='references'></a>

## References

*Reference: Please include all relevant links (git, video, etc)*

7. Does it include all deliverables (3)
	- git with codes or notebooks
	- writeup (you can consider notebook as a writeup if the notebook contains all needed contents and explanation)
	- demo clips
	- proper quote or reference
    
Kauten, C. (2018). Super Mario Bros for OpenAI Gym. GitHub. Retrieved from https://github.com/Kautenja/gym-super-mario-bros

Nintendo. (1985). Super Mario Bros. Instruction Manual. Nintendo of America Inc. Retrieved from [https://www.nintendo.co.jp/clv/manuals/en/pdf/CLV-P-NAAAE.pdf]

NathanGavenski. (2023). Comment on issue #128 in Kautenja/gym-super-mario-bros repository. GitHub. Retrieved from https://github.com/Kautenja/gym-super-mario-bros/issues/128#issuecomment-1954019091

Kundu, S. (2023, October 2). Train AI to beat Super Mario Bros! || Reinforcement learning completely from scratch [Video]. YouTube. https://www.youtube.com/watch?v=_gmQZToTMac


